In [ ]:
pip install git+https://github.com/boilingdata/py-boilingdata@v0.2.7

In [ ]:
%env BD_USERNAME=****

In [ ]:
%env BD_PASSWORD=****

In [ ]:
import pandas as pd
from py_boilingdata import BoilingData

boiling = BoilingData()
await boiling.connect()

async def bd(sql):
    return await boiling.execute(sql)

In [ ]:
await boiling.execute('SHOW TABLES')

In [ ]:
df = pd.DataFrame(await boiling.execute("SELECT first_name, email FROM parquet_scan('s3://boilingdata-demo/test.parquet') LIMIT 10"))
print(df)

In [ ]:
pd.DataFrame(await boiling.execute("SELECT * FROM parquet_scan('s3://boilingdata-demo/test.parquet') LIMIT 12"))

In [ ]:
await boiling.close()